In [37]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [38]:
df = pd.read_csv('../data/aden_golfe_matrix_area.csv',sep=';')


In [39]:
def month(quinzine):
    if(str(quinzine).split('.', 1 )[1]=='5'):
        if len(str(int(str(quinzine).split('.', 1 )[0])+1))==1:
            return '0'+str(int(str(quinzine).split('.', 1 )[0])+1)
        else:
            return str(int(str(quinzine).split('.', 1 )[0])+1)
    else:
        if len(str(quinzine).split('.', 1 )[0])==1:
            return '0'+str(quinzine).split('.', 1 )[0]
        else:
            return str(quinzine).split('.', 1 )[0]
        
def day(quinzine):
    if(quinzine%2==0):
        return '15'
    else:
        return '01'
    
print day(2) 
print day(1) 
df['month'] =df.apply(lambda row: month(row['2_week']/2), axis=1)
df['day'] =df.apply(lambda row: day(row['2_week']), axis=1)
df['date'] =df.apply(lambda row: str(row['Years'])+'-'+str(row['month'])+'-'+str(row['day']), axis=1)

df['date']=df.apply(lambda row: datetime.datetime.strptime(row['date'], "%Y-%m-%d"), axis=1)

#df['date']=datetime.datetime.strptime(df['date'], "%Y-%m-%d")

print df[df['count']!=0]
#print len(df)
#print df.dtypes

15
01
       2_week  Years  count  lat_area  lon_area month day       date
980         3   2008      2         4        20    02  01 2008-02-01
2197        6   2008      1         3        21    03  15 2008-03-15
2226        6   2008      1         4        18    03  15 2008-03-15
2291        6   2008      1         6        19    03  15 2008-03-15
2672        7   2008      1         5        16    04  01 2008-04-01
2674        7   2008      1         5        18    04  01 2008-04-01
2736        7   2008      1         7        16    04  01 2008-04-01
3056        8   2008      1         4        16    04  15 2008-04-15
3057        8   2008      1         4        17    04  15 2008-04-15
3122        8   2008      1         6        18    04  15 2008-04-15
3888       10   2008      1         4        16    05  15 2008-05-15
3891       10   2008      1         4        19    05  15 2008-05-15
3918       10   2008      1         5        14    05  15 2008-05-15
3920       10   2008      1 

In [40]:
features=['2_week','Years','month','day']
target='count'

In [41]:
dfTrain=df[df['Years']!=2014]
dfTest=df[df['Years']==2014]
print len(dfTrain[dfTrain['count']!=0])
print len(dfTest[dfTest['count']!=0])

print len(dfTrain)
print len(dfTest)

337
10
59904
9984


# Forest

In [46]:
forest = RandomForestClassifier(n_jobs=1)
forest=forest.fit(dfTrain[features], dfTrain[target])

In [47]:
predictions=forest.predict(dfTest[features])
print predictions.sum()

0


In [48]:
pd.crosstab(dfTest[target],predictions,rownames=['actual'], colnames=['preds'])

preds,0
actual,
0,9974
1,10


# Compare area

In [45]:
dfTest['prediction']=predictions
dfTest[(dfTest[target]!=0) | (dfTest['prediction']!=0)].sort('2_week')

/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,2_week,Years,count,lat_area,lon_area,month,day,date,prediction
60601,2,2014,1,8,25,01,15,2014-01-15,0
60838,3,2014,1,3,6,02,01,2014-02-01,0
61783,5,2014,1,6,23,03,01,2014-03-01,0
62149,6,2014,1,5,5,03,15,2014-03-15,0
62502,7,2014,1,3,6,04,01,2014-04-01,0
62534,7,2014,1,4,6,04,01,2014-04-01,0
62619,7,2014,1,6,27,04,01,2014-04-01,0
62918,8,2014,1,3,6,04,15,2014-04-15,0
66277,16,2014,1,4,5,08,15,2014-08-15,0
69731,24,2014,1,8,3,12,15,2014-12-15,0
